In [3]:
%pip install yahooquery
%pip install sklearn

  Using cached yahooquery-2.3.7-py3-none-any.whl (52 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gianm\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Users\gianm\AppData\Local\Programs\Python\Python310\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\gianm\\AppData\\Local\\Temp\\pip-install-z5k80lg0\\sklearn_94b3516c9ba34e5bb00ba4745d1fa1c3\\setup.py'"'"'; __file__='"'"'C:\\Users\\gianm\\AppData\\Local\\Temp\\pip-install-z5k80lg0\\sklearn_94b3516c9ba34e5bb00ba4745d1fa1c3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\gianm\AppData\Local\Temp\pip-pip-egg-info-n4_6umsf'
         cwd: C:\Users\gianm\AppData\Local\Temp\pip-install-z5k80lg0\sklearn_94b3516c9ba34e5bb00ba4745d1fa1c3\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  

In [8]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler

symbols = [ 'VFC',  'INLX', 'IONM']
sectorDict = {}
snpClosing = pd.DataFrame()  
scaler = StandardScaler()

for sector in symbols:
    snpSect = yf.download(tickers=sector,start="2019-01-01" , end="2021-12-31")
    snpClosingSymbol = snpSect['Close'].reset_index()
    snpClosingSymbol['Date'] = snpClosingSymbol['Date'].dt.strftime('%Y-%m-%d')
    snpClosingSymbol[sector] = snpClosingSymbol[sector].round(2).dropna(inplace=True)
    snpClosingSymbol.set_index('Date', inplace=True)

    if snpClosing.empty:
        snpClosing = snpClosingSymbol
    else:
        snpClosing = pd.merge(snpClosing, snpClosingSymbol, on='Date', how='outer')

snpClosing = scaler.fit_transform(snpClosing)

#snpClosing.set_index('Date', inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
c:\Users\gianm\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\gianm\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\gianm\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


VFC	    VF Corporation	        Apparel Manufacturing	Consumer Cyclical 

INLX	Intellinetics,   Inc.	Software—Application	Technology

IONM	Assure Holdings Corp.	Medical Care Facilities	Healthcare

In [12]:
print(snpClosing)

[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]


In [1]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA()
principal_components = pca.fit_transform(snpClosing)

pca_df = pd.DataFrame(data=principal_components, columns=[f"PC{i+1}" for i in range(len(pca.explained_variance_ratio_))], index=snpClosing.index)
explained_variance = pca.explained_variance_ratio_

print("Explained Variance Ratio:", explained_variance)

# Visualize explained variance
plt.figure(figsize=(8, 5))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, align='center', 
        label='Individual Explained Variance')
plt.step(range(1, len(explained_variance) + 1), explained_variance.cumsum(), where='mid', 
         label='Cumulative Explained Variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.title('Explained Variance by Principal Components')
plt.legend(loc='best')
plt.show()


ModuleNotFoundError: No module named 'sklearn'